# Importations and paths

In [1]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")   # remonte de notebooks → project_root
SRC_PATH = os.path.join(PROJECT_ROOT, "src")

if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

print("SRC path added:", SRC_PATH)

SRC path added: /Users/cassiedoguet/Library/CloudStorage/OneDrive-Personnel/Documents/ESILV_A5S1_V2/CV and deep learning/Project/WAID_CV_project/detection-and-identification-of-wildlife-populations-from-drone-images/src


In [2]:
import time
import torch
import numpy as np

from torch.utils.data import DataLoader
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

from torchmetrics.detection.mean_ap import MeanAveragePrecision

from data.dataset import WAIDDataset
from data.augmentations import get_val_transforms


/Users/cassiedoguet/Library/CloudStorage/OneDrive-Personnel/Documents/ESILV_A5S1_V2/CV and deep learning/Project/WAID_CV_project/detection-and-identification-of-wildlife-populations-from-drone-images/.venv/lib/python3.12/site-packages/albumentations/check_version.py:147: UserWarning: Error fetching version info <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)>
  data = fetch_version_info()


In [3]:
PROJECT_ROOT = os.path.abspath("..")
DATA_DIR = os.path.join(PROJECT_ROOT, "data")
RAW_DATA_DIR = os.path.join(DATA_DIR, "raw")

IMAGE_DIR = os.path.join(RAW_DATA_DIR, "images")
ANNOTATION_DIR = os.path.join(RAW_DATA_DIR, "annotations")
CLASSES_PATH = os.path.join(DATA_DIR, "classes.txt")

with open(CLASSES_PATH) as f:
    CLASS_NAMES = [line.strip() for line in f if line.strip()]

NUM_CLASSES = len(CLASS_NAMES) + 1  # + background
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("Device:", DEVICE)
print("Nb classes:", len(CLASS_NAMES))

Device: cpu
Nb classes: 6


In [4]:
test_image_dir = os.path.join(IMAGE_DIR, "test")
test_ann_dir = os.path.join(ANNOTATION_DIR, "test")

test_image_files = sorted([
    f for f in os.listdir(test_image_dir)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
])

print("Test images:", len(test_image_files))

Test images: 1437


In [5]:
def collate_fn(batch):
    images = [item["image"] for item in batch]
    targets = []
    for item in batch:
        targets.append({
            "bboxes": item["bboxes"],
            "labels": item["labels"],
            "image_size": item["image_size"]
        })
    return images, targets

In [6]:
test_dataset = WAIDDataset(
    image_dir=test_image_dir,
    annotation_dir=test_ann_dir,
    image_files=test_image_files,
    num_classes=len(CLASS_NAMES),
    transforms=get_val_transforms()  # ok pour test (pas d'augmentation)
)

test_loader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0,   # ⬅️ IMPORTANT
    collate_fn=collate_fn
)


/Users/cassiedoguet/Library/CloudStorage/OneDrive-Personnel/Documents/ESILV_A5S1_V2/CV and deep learning/Project/WAID_CV_project/detection-and-identification-of-wildlife-populations-from-drone-images/.venv/lib/python3.12/site-packages/albumentations/core/composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()


In [7]:
model = fasterrcnn_resnet50_fpn(
    weights=None,
    weights_backbone=None
)

in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, NUM_CLASSES)

ckpt_path = "../outputs/models/faster_rcnn_waid.pth"
state = torch.load(ckpt_path, map_location="cpu")
model.load_state_dict(state)

model = model.to(DEVICE)
model.eval()

print("Loaded:", ckpt_path)


Loaded: ../outputs/models/faster_rcnn_waid.pth


In [8]:
def box_iou(boxA, boxB):
    # box: [xmin, ymin, xmax, ymax]
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    inter = max(0, xB - xA) * max(0, yB - yA)
    areaA = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    areaB = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    union = areaA + areaB - inter
    return inter / union if union > 0 else 0

In [9]:
@torch.no_grad()
def evaluate_detection_simple(
    model,
    loader,
    device,
    iou_thresh=0.5,
    score_thresh=0.3
):
    model.eval()

    TP = FP = FN = 0

    for images, targets in loader:
        images = [img.to(device) for img in images]
        outputs = model(images)

        for out, tgt in zip(outputs, targets):
            gt_boxes = tgt["bboxes"].cpu().numpy()
            gt_labels = tgt["labels"].cpu().numpy()

            keep = out["scores"].cpu().numpy() >= score_thresh
            pred_boxes = out["boxes"].cpu().numpy()[keep]
            pred_labels = out["labels"].cpu().numpy()[keep]

            matched_gt = set()

            for pb, pl in zip(pred_boxes, pred_labels):
                best_iou = 0
                best_gt = -1

                for i, (gb, gl) in enumerate(zip(gt_boxes, gt_labels)):
                    if i in matched_gt or pl != gl:
                        continue
                    iou = box_iou(pb, gb)
                    if iou > best_iou:
                        best_iou = iou
                        best_gt = i

                if best_iou >= iou_thresh:
                    TP += 1
                    matched_gt.add(best_gt)
                else:
                    FP += 1

            FN += len(gt_boxes) - len(matched_gt)

    precision = TP / (TP + FP + 1e-8)
    recall = TP / (TP + FN + 1e-8)
    f1 = 2 * precision * recall / (precision + recall + 1e-8)

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "TP": TP,
        "FP": FP,
        "FN": FN,
    }


In [13]:
import cv2

img_path = "../data/raw/images/test/0cba95f9-b89e-4034-8543-86276b057dae_jpg.rf.c8e47f70d213aae466cf3a59fcfd6c0bc.jpg"

image = cv2.imread(img_path)

if image is None:
    raise FileNotFoundError(f"OpenCV cannot read image: {img_path}")

image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

[ WARN:0@107.928] global loadsave.cpp:278 findDecoder imread_('../data/raw/images/test/0cba95f9-b89e-4034-8543-86276b057dae_jpg.rf.c8e47f70d213aae466cf3a59fcfd6c0bc.jpg'): can't open/read file: check file path/integrity


FileNotFoundError: OpenCV cannot read image: ../data/raw/images/test/0cba95f9-b89e-4034-8543-86276b057dae_jpg.rf.c8e47f70d213aae466cf3a59fcfd6c0bc.jpg

In [12]:
metrics = evaluate_detection_simple(
    model,
    test_loader,
    DEVICE,
    iou_thresh=0.5,
    score_thresh=0.3
)

for k, v in metrics.items():
    print(f"{k:10s}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}")


error: OpenCV(4.13.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
# res["map_per_class"] : tensor [C]
map_pc = res.get("map_per_class", None)
mar_pc = res.get("mar_100_per_class", None)

if map_pc is not None:
    map_pc = map_pc.cpu().numpy()
    order = np.argsort(-np.nan_to_num(map_pc, nan=-1))

    print("\nTop classes by AP:")
    for idx in order[:10]:
        print(f"{CLASS_NAMES[idx]:25s} AP={map_pc[idx]:.3f}")

    print("\nWorst classes by AP:")
    for idx in order[-10:]:
        print(f"{CLASS_NAMES[idx]:25s} AP={map_pc[idx]:.3f}")


In [ ]:
import matplotlib.pyplot as plt
import torchvision

def draw_boxes(img, boxes, labels, scores=None, class_names=None):
    # img: tensor [C,H,W] in [0,1]
    img_uint8 = (img * 255).byte()
    texts = []
    for i in range(len(boxes)):
        lab = int(labels[i])
        name = class_names[lab] if class_names is not None else str(lab)
        if scores is not None:
            texts.append(f"{name}:{scores[i]:.2f}")
        else:
            texts.append(name)

    drawn = torchvision.utils.draw_bounding_boxes(
        img_uint8,
        boxes=boxes.cpu(),
        labels=texts,
        width=2
    )
    return drawn

@torch.no_grad()
def show_predictions(model, dataset, idxs, device, score_thresh=0.3):
    model.eval()
    for idx in idxs:
        item = dataset[idx]
        img = item["image"].to(device)
        gt_boxes = item["bboxes"]
        gt_labels = item["labels"]

        out = model([img])[0]
        keep = out["scores"] >= score_thresh

        pred_boxes = out["boxes"][keep]
        pred_labels = out["labels"][keep]
        pred_scores = out["scores"][keep]

        # draw GT then Pred
        img_gt = draw_boxes(item["image"], gt_boxes, gt_labels, class_names=CLASS_NAMES)
        img_pr = draw_boxes(item["image"], pred_boxes, pred_labels, pred_scores, class_names=CLASS_NAMES)

        plt.figure(figsize=(14,6))
        plt.subplot(1,2,1); plt.title(f"GT idx={idx}"); plt.imshow(img_gt.permute(1,2,0)); plt.axis("off")
        plt.subplot(1,2,2); plt.title(f"Pred (thr={score_thresh})"); plt.imshow(img_pr.permute(1,2,0)); plt.axis("off")
        plt.show()

# exemple
show_predictions(model, test_dataset, idxs=[0, 5, 12], device=DEVICE, score_thresh=0.3)

In [ ]:
@torch.no_grad()
def benchmark_inference(model, dataset, device, n=50):
    model.eval()
    idxs = np.random.choice(len(dataset), size=min(n, len(dataset)), replace=False)

    # warmup
    for _ in range(5):
        img = dataset[int(idxs[0])]["image"].to(device)
        _ = model([img])

    if device == "cuda":
        torch.cuda.synchronize()

    t0 = time.time()
    for i in idxs:
        img = dataset[int(i)]["image"].to(device)
        _ = model([img])
    if device == "cuda":
        torch.cuda.synchronize()

    elapsed = time.time() - t0
    per_img = elapsed / len(idxs)
    print(f"{len(idxs)} images | {per_img*1000:.1f} ms/img | {1/per_img:.2f} FPS")

benchmark_inference(model, test_dataset, DEVICE, n=100)

In [ ]:
def full_eval(model, name):
    res, elapsed, n_images = evaluate_map(model, test_loader, DEVICE, score_thresh=0.05)
    print(f"\n===== {name} =====")
    print(f"Time: {elapsed/n_images:.3f}s/img")
    print("mAP:", float(res["map"]), "| AP50:", float(res["map_50"]), "| AP75:", float(res["map_75"]))
    return res

# res_frcnn = full_eval(model, "Faster R-CNN")
# res_ssd   = full_eval(ssd_model, "SSD")
# res_yolo  = full_eval(yolo_model, "YOLO")